In [23]:
import lightning as L
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch
from torch.utils.data import DataLoader
from lightning import Trainer
from icecream import ic


cuda = torch.cuda.is_available()
if cuda:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [22]:
train_x = torch.load('data/train_x.pt')
train_y = torch.load('data/train_y.pt')
test_x  =  torch.load('data/test_x.pt')
test_y  =  torch.load('data/test_y.pt')

train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

(torch.Size([100000, 100]),
 torch.Size([10000, 100]),
 torch.Size([100000]),
 torch.Size([10000]))

In [30]:
class MyModel(L.LightningModule):
    def __init__(self, partition_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(partition_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


In [ ]:
model = MyModel(train_x.shape[-1])
trainer = Trainer(max_epochs=10, accelerator="auto", devices=1)
trainer.fit(model, train_dataloaders=train_loader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | Sequential | 6.5 K  | train
1 | loss_fn | MSELoss    | 0      | train
-----------------------------------------------
6.5 K     Trainable params
0         Non-trainable params
6.5 K     Total params
0.026     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Epoch 1:   4%|▍         | 32/782 [00:00<00:04, 180.48it/s, v_num=4] 

c:\Users\alvis\.conda\envs\CF\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2:  22%|██▏       | 173/782 [00:00<00:03, 182.40it/s, v_num=4]